<a href="https://colab.research.google.com/github/obeabi/ProjectPortfolio/blob/master/TA_Weekly_Entry_etf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install --upgrade yfinance
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=a30411f64487146ec69df5450d7cae1f8bf103f7737af77ab558f6683900b009
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [2]:
import yfinance as yf
print(yf.__version__)
import pandas as pd
import numpy as np
import time
import ta
print("Libraries Installed!")

0.2.54
Libraries Installed!


In [3]:
# List of ETFs to analyze
df = pd.read_csv('etf_list.csv')
etfs = df['ETF'].to_list()
print(etfs)

['UNG', 'DRAG', 'UNL', 'CQQQ', 'ECNS', 'GXC', 'EWP', 'EWO', 'CNXT', 'EUFN', 'IHE', 'EWI', 'MORT', 'EFNL', 'IYK', 'PPH', 'XLP', 'VDC', 'KXI', 'IEV', 'EWH', 'IEUR', 'EWL', 'VGK', 'SPEU', 'VNM', 'VFMV', 'AIVI', 'EWK', 'IXJ', 'XLV', 'CNYA', 'BIDD', 'SMOG', 'EIRL', 'SCJ', 'IYH', 'VHT']


In [4]:
# Function to fetch historical weekly data
def get_weekly_data(ticker):
  try:
      df = yf.download(ticker, period="2y", interval="1wk")
      df['20_week_SMA'] = df['Close'].rolling(window=20).mean()
      df['50_week_SMA'] = df['Close'].rolling(window=50).mean()
      df['RSI'] = compute_rsi(df['Close'])
      df['ATR'] = compute_atr(df, 14)
      df['OBV'] = compute_obv(df)
      df['10_week_avg_volume'] = df['Volume'].rolling(window=10).mean()
      df['ma'] = calculate_ma(df['RSI'])
      df['upper_band'], df['lower_band'] = calculate_bollinger_bands(df['RSI'])
      return df
  except Exception as e:
      print("There is an error getting weekly data", e)

# Function to compute RSI
def compute_rsi(series, period=14):
  try:
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))
  except Exception as e:
        print("Something went wrong while computing the RSI:", e)
        return None

def calculate_ma(data, length=14, ma_type="WMA"):
    if ma_type == "SMA":
        return data.rolling(window=length).mean()
    elif ma_type == "EMA":
        return data.ewm(span=length, adjust=False).mean()
    elif ma_type == "WMA":
        weights = np.arange(1, length+1)
        return data.rolling(length).apply(lambda x: np.dot(x, weights)/weights.sum(), raw=True)
    elif ma_type == "VWMA":
        return ta.volume_weighted_average_price(data, length)

def calculate_bollinger_bands(data, length=14, std_dev=2.0):
    sma = data.rolling(window=length).mean()
    std_dev = data.rolling(window=length).std()
    upper_band = sma + (std_dev * std_dev)
    lower_band = sma - (std_dev * std_dev)
    return upper_band, lower_band

# Function to compute ATR (Average True Range)
def compute_atr(df, period=14):
  try:
    df['High-Low'] = df['High'] - df['Low']
    df['High-Close'] = abs(df['High'] - df['Close'].shift(1))
    df['Low-Close'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    return df['TR'].rolling(window=period).mean()
  except Exception as e:
      print("Something went wrong whilecomputing the ATR", e)


# Function to compute On-Balance Volume (OBV)
def compute_obv(df):
  try:
    # Calculate daily price change: 1 if price is up, -1 if down, 0 if unchanged
    price_change = df['Close'].diff()

    # Use price change to decide whether to add or subtract volume
    obv = (price_change > 0).astype(int) * df['Volume']  # Volume when price goes up
    obv -= (price_change < 0).astype(int) * df['Volume']  # Volume when price goes down

    # We accumulate the OBV by taking the cumulative sum of the volume changes
    obv = obv.cumsum()

    return obv
  except Exception as e:
      print("Something went wrong while computing the OBV", e)


# Function to calculate risk-reward ratio
def calculate_risk_reward(df):
  try:
    if df.empty or len(df) < 20:  # Ensure there are enough data points
        return np.nan

    latest_price = df['Close'].iloc[-1].iloc[0]

    # Use the ATR for setting support level
    atr = df['ATR'].iloc[-1]  # Latest ATR value
    atr_multiple = 1.25  # You can adjust this multiplier based on your strategy

    # Calculate the support level using the ATR
    trailing = atr * atr_multiple
    support_level = latest_price - trailing
    resistance_level = latest_price + (2*trailing)
    risk = latest_price- support_level
    reward = resistance_level - latest_price

    # Ensure risk is greater than zero before division
    if risk > 0:
        risk_reward_ratio = reward / risk
        return risk_reward_ratio if risk_reward_ratio > 0 else np.nan , support_level, resistance_level, latest_price, trailing
    else:
        return np.nan,np.nan, np.nan, np.nan, np.nan
  except Exception as e:
      print("Something went wrong while computing the reward-risk ratio", e)

# Function to fetch daily data
def get_daily_data(ticker):
    df = yf.download(ticker, period="1y", interval="1d")
    df['20_day_SMA'] = df['Close'].rolling(window=20).mean()
    df['20_day_EMA'] = df['Close'].ewm(span=20, adjust=False).mean()
    df["Distance_EMA"] = (df['Close']/ df['Close'].ewm(span=20, adjust=False).mean() ) - 1
    df['RSI'] = compute_rsi(df['Close'])
    df['ATR'] = compute_atr(df, 20)
    df['ma'] = calculate_ma(df['RSI'])
    df['upper_band'], df['lower_band'] = calculate_bollinger_bands(df['RSI'])
    return df

# Function to check weekly trend
def is_weekly_trend_bullish(df):
    if df.empty:
        return False

    latest_price = df['Close'].iloc[-1].iloc[0]
    latest_sma = df['20_week_SMA'].iloc[-1]
    latest_rsi = df['RSI'].iloc[-1]

    return (latest_price > latest_sma) and (latest_rsi > 50)

# Function to check daily entry signal
def is_daily_entry_signal(df):
    if df.empty:
        return False

    latest_price = df['Close'].iloc[-1].iloc[0]
    prev_price = df['Close'].iloc[-2].iloc[0]
    latest_sma = df['20_day_SMA'].iloc[-1]
    latest_rsi = df['RSI'].iloc[-1]
    latest_distance_20ema = df['Distance_EMA'].iloc[-1]

    # Look for a breakout above 20-day SMA & RSI > 50
    return (latest_price > latest_sma) and (latest_rsi > 50) and (latest_rsi < 75)

# Check entry conditions
def check_entry_conditions(tickers):
    results = []
    for ticker in tickers:
      df = get_daily_data(ticker)
      latest_price = df['Close'].iloc[-1].iloc[0]
      prev_price = df['Close'].iloc[-2].iloc[0]
      latest_sma = df['20_day_SMA'].iloc[-1]
      latest_rsi = df['RSI'].iloc[-1]
      latest_distance_20ema = df['Distance_EMA'].iloc[-1]

      if (latest_price >= latest_sma) and (latest_rsi< 70) :
        if (latest_distance_20ema <= 0.02):
          entry_signal= "Enter full position"
        elif latest_distance_20ema <= 0.04:
          entry_signal = "Enter 50% now and rest later"
        else:
          entry_signal= "Wait for pullback"
      else:
        entry_signal = "RSI is overbought- Wait for pullback"
      results.append([ticker, entry_signal])
    # Convert results to DataFrame
    df_results = pd.DataFrame(results, columns=["ETF", "Entry_Signal"])
    return df_results

# Multi-timeframe strategy check returning a DataFrame
def check_mtf_entry(tickers):
    results = []

    for ticker in tickers:
        weekly_df = get_weekly_data(ticker)
        daily_df = get_daily_data(ticker)

        if is_weekly_trend_bullish(weekly_df):
            if is_daily_entry_signal(daily_df):
                entry_signal = "Entry Confirmed ✅"
            else:
                entry_signal = "No Entry Yet ⏳"
        else:
            entry_signal = "Weekly Trend Not Bullish ❌"

        results.append([ticker, entry_signal])
        time.sleep(2)  # Add a delay of 1 second between requests

    # Convert results to DataFrame
    df_results = pd.DataFrame(results, columns=["ETF", "Entry_Signal"])
    return df_results

In [8]:
# Apply TA filters and prioritize ETFs
results = []
for etf in etfs:
  df =get_weekly_data(etf)
  price = df['Close'].iloc[-1].iloc[0]
  above_20SMA = price > df['20_week_SMA'].iloc[-1]
  above_50SMA = price > df['50_week_SMA'].iloc[-1]
  rsi_ok = 40 <= df['RSI'].iloc[-1] <= 70 # Not overbought or oversold
  volume_ok = df['Volume'].iloc[-1] > df['10_week_avg_volume'].iloc[-1] # Institutional interest
  volume_ok = volume_ok.iloc[0]

  # Calculate the OBV Moving Average
  df['OBV_EMA'] = df['OBV'].ewm(span=10, adjust=False).mean()

  # OBV trending up if current OBV is above the 20-period EMA
  obv_trending_up = df['OBV'].iloc[-1] > df['OBV_EMA'].iloc[-1]
  # obv_trending_up = df['OBV'].iloc[-1] > df['OBV'].iloc[-5] # OBV increasing over last 5 weeks

  # OBV trending down if current OBV is below the 20-period EMA
  obv_trending_down = df['OBV'].iloc[-1] < df['OBV_EMA'].iloc[-1]

  trend_ok = above_20SMA and above_50SMA


  if trend_ok and rsi_ok and (volume_ok and obv_trending_up):
    print(f" {etf} passes the first check on weekly timeframe!")
    results.append({"ETF": etf })
  else:
    print(f" {etf} does not pass the first check on weekly timeframe!")
  time.sleep(2)  # Add a delay of 1 second between requests


# Multi-time frame entry Check
df_results = pd.DataFrame(results).dropna()
etfs_to_check = df_results['ETF'].tolist()
df_signals = check_mtf_entry(etfs_to_check)

df_signals.head()



[*********************100%***********************]  1 of 1 completed


 UNG does not pass the first check on weekly timeframe!


[*********************100%***********************]  1 of 1 completed


 DRAG does not pass the first check on weekly timeframe!


[*********************100%***********************]  1 of 1 completed


...

 IYH does not pass the first check on weekly timeframe!


[*********************100%***********************]  1 of 1 completed


 VHT does not pass the first check on weekly timeframe!


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,ETF,Entry_Signal
0,CQQQ,No Entry Yet ⏳
1,ECNS,No Entry Yet ⏳


## Generate buy list

In [6]:
df_final = df_signals[df_signals['Entry_Signal'] =="Entry Confirmed ✅"]
final_etfs_to_check = df_final['ETF'].tolist()

buy_list = check_entry_conditions(final_etfs_to_check)

buy_list

,ETF,Entry_Signal


In [7]:
# Apply TA filters and prioritize ETFs
results = []
buy_list = buy_list[buy_list['Entry_Signal'] !="RSI is overbought- Wait for pullback"]
for etf in buy_list['ETF'].to_list():
  df =get_weekly_data(etf)
  price = df['Close'].iloc[-1].iloc[0]
  above_20SMA = price > df['20_week_SMA'].iloc[-1]


  if  above_20SMA :
    rr_ratio,support_level, resistance_level, latest_price,trail = calculate_risk_reward(df)
    results.append({"ETF": etf ,"Risk-Reward": rr_ratio, "Support": support_level, "Resistance": resistance_level, "Current Price": latest_price, "Trail Price": trail})
  time.sleep(2)  # Add a delay of 1 second between requests


# Sort ETFs by highest risk-to-reward ratio
df_results = pd.DataFrame(results).dropna().sort_values(by="Risk-Reward", ascending=False).reset_index(drop = True)
df_results


KeyError: 'Risk-Reward'